# Lab 05: Gene Expression

---
author: Your Name Here
date: March 4, 2024
embed-resources: true
---

Submit this template to Canvas for full credit!

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from joblib import dump
from sklearn.tree import DecisionTreeRegressor

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
genes_train = pd.read_csv("https://cs307.org/lab-05/data/genes-train.csv")
# create X and y for train
X_train = genes_train.drop("y", axis=1)
y_train = genes_train["y"]

In [4]:
X_train

,lvfyodfjhd,chlxjpxmff,otpruhfgcr,oehlcawgpf,qoizufnzxh,zkyigilmjr,qjhqeatvex,hlvjxolgkj,lrwdgjvoau,dwkbxacpab,...,baivhgmwgj,forscbxhfb,qqoqizjjqj,deynskqfbg,effadqkxyr,eofvzakuue,wismhvmnxb,cvjwglhnga,jsdmcbejne,cvoxpagxgt
0,1.523299,19.404329,0.003386,1.926715,1.301010,2.050354,4.251543,2.212030,5.086263,5.486965,...,1.336388,1.971657,73.573000,2.698167,10.228457,447.017647,1.845999,0.656854,4.641398,0.781940
1,2.125917,79.447265,0.098064,1.756492,88.349079,0.404541,4.361385,110.207743,4.232306,5.335660,...,3.751518,1.344905,10.645052,0.410613,9.216523,78.347809,0.224902,1.567216,8.906766,0.105189
2,2.301169,89.695207,0.099854,0.459814,65.344218,0.106759,3.194797,134.950376,3.997683,5.600865,...,0.628325,3.978908,31.431066,1.687640,9.344670,23.715733,3.062889,0.000000,1.210128,0.747997
3,1.748618,49.998491,0.067186,0.965176,33.704983,4.959406,3.285966,2.696591,2.715285,5.333225,...,5.088898,4.466364,56.652506,1.344890,8.942019,127.498795,1.143567,1.933546,6.783099,1.120372
4,1.666441,13.239583,0.109528,0.000000,97.397923,2.208333,3.111847,22.831821,6.380585,5.741707,...,2.623644,2.213965,83.047111,0.148668,9.313350,49.571299,1.032066,0.000000,29.263633,0.296164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2.117729,6.385713,0.109704,0.966220,206.352017,0.217017,5.701002,257.074823,9.586723,5.351280,...,2.143226,0.430624,246.244494,0.033559,11.104503,36.874867,2.022519,0.582008,0.586234,0.805512
896,1.921916,4.848924,0.049569,2.332017,11.699854,0.404328,4.555099,121.503267,2.118825,5.541698,...,2.558042,0.000000,59.804792,2.281729,10.910872,166.735774,2.183504,0.000000,24.536640,0.342456
897,1.714503,35.534482,0.070890,3.021258,26.574131,2.068426,6.185746,30.240624,2.288780,5.539048,...,3.157934,0.000000,0.276544,4.807311,11.258259,236.580225,0.466819,1.777398,5.399876,0.589704
898,1.850003,13.045160,0.081111,0.087547,3.688737,2.051105,4.495955,41.987975,8.012388,5.580206,...,3.894236,1.470798,23.657785,4.598416,10.353938,4.010867,0.726227,0.000000,1.114646,0.340193


In [5]:
y_train

0      36
1      42
2      44
3      51
4      40
       ..
895    42
896    36
897    40
898    35
899    38
Name: y, Length: 900, dtype: int64

In [6]:
X_train.dtypes

lvfyodfjhd    float64
chlxjpxmff    float64
otpruhfgcr    float64
oehlcawgpf    float64
qoizufnzxh    float64
               ...   
eofvzakuue    float64
wismhvmnxb    float64
cvjwglhnga    float64
jsdmcbejne    float64
cvoxpagxgt    float64
Length: 750, dtype: object

In [7]:
# split the data into numerical and categorical
numeric_features = X_train.select_dtypes(include=["float64"]).columns
categorical_features = X_train.select_dtypes(include=["object"]).columns

In [8]:
# define how to handle missing data and scale features for numeric and categorical features
numeric_transformer = Pipeline(
    steps=[
        ("Iterative Imputer", IterativeImputer(estimator=LinearRegression())),
        ("Standardization", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("Modal Imputer", SimpleImputer(strategy="most_frequent")),
        ("One-Hot Encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

In [9]:
# create general preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("Numeric Transformer", numeric_transformer, numeric_features),
        ("Categorical Transformer", categorical_transformer, categorical_features),
    ],
    remainder="drop",
)

In [10]:
'''model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', LinearRegression())])'''

"model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),\n                            ('regressor', LinearRegression())])"

In [11]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomialfeatures', PolynomialFeatures()),
    ('regressor', LinearRegression())
])

# 创建参数网格，仅调整多项式特征的度数


In [15]:
param_grid = {
    "polynomialfeatures__degree": [1,2]  # 例如设置多项式的最大度数
}

In [16]:
'''param_grid = {
    "regressor__n_neighbors": range(1, 15),  # set the range of k
    # define how the neighbors contribute, equally or have different weights based on distance
    "regressor__weights": ["uniform", "distance"],
    # distance metrics to be used for calculating the proximity between data points
    "regressor__metric": ["euclidean", "manhattan"]
}'''

'param_grid = {\n    "regressor__n_neighbors": range(1, 15),  # set the range of k\n    # define how the neighbors contribute, equally or have different weights based on distance\n    "regressor__weights": ["uniform", "distance"],\n    # distance metrics to be used for calculating the proximity between data points\n    "regressor__metric": ["euclidean", "manhattan"]\n}'

In [17]:
'''param_grid = {
    "regressor__max_depth": range(1, 15),  # 例如设置树的深度范围
    #"regressor__min_samples_split": range(2, 10),  # 设置分割内部节点所需的最小样本数
    #"regressor__min_samples_leaf": range(1, 10),  # 设置叶节点所需的最小样本数
    #"regressor__max_features": ["auto", "sqrt", "log2"]  # 设置最佳分割时考虑的特征数量
}'''


'param_grid = {\n    "regressor__max_depth": range(1, 15),  # 例如设置树的深度范围\n    #"regressor__min_samples_split": range(2, 10),  # 设置分割内部节点所需的最小样本数\n    #"regressor__min_samples_leaf": range(1, 10),  # 设置叶节点所需的最小样本数\n    #"regressor__max_features": ["auto", "sqrt", "log2"]  # 设置最佳分割时考虑的特征数量\n}'

In [18]:
# setup grid search with cross-validation
grid_search = GridSearchCV(model_pipeline, param_grid,
                           cv=5, scoring="neg_mean_squared_error")

In [19]:
grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
dump(grid_search, "gene-expression.joblib")

['gene-expression.joblib']

In [ ]:
print("Best parameters found:", grid_search.best_params_)
print("Lowest RMSE found:", np.sqrt(-grid_search.best_score_))

Best parameters found: {'regressor__max_depth': 5}
Lowest RMSE found: 5.0627214058622725
